<a href="https://colab.research.google.com/github/chacha86/pythonai/blob/main/%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1%ED%9A%8C%EA%B7%80(%EB%B6%84%EB%A5%98)_%EB%AC%BC%EA%B3%A0%EA%B8%B0_%EC%96%B4%EC%A2%85_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

fish = pd.read_csv('/content/Fish.csv')

In [ ]:
## Bream, Smelt
## 길이와 무게 특성만 고려

bream_smelt = fish[(fish['Species'] == 'Bream') | (fish['Species'] == 'Smelt')]

bream_smelt_train = bream_smelt[['Length1', 'Weight']]
bream_smelt_target = bream_smelt['Species']

## 학습데이터와 테스트 데이터 0.7, 0.3 비율로 만들어주세요.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

trd, tsd, trt, tst = train_test_split(bream_smelt_train, bream_smelt_target, train_size=0.7, random_state=42)

ss = StandardScaler()

scaled_trd = ss.fit_transform(trd)
scaled_tsd = ss.transform(tsd)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(scaled_trd, trt)
lr.score(scaled_trd, trt), lr.score(scaled_tsd, tst)


(1.0, 1.0)